## Nuclear Masses

In [1]:
import ultranest
import scipy.stats
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import read as rd


from ultranest.plot import cornerplot

### Bayesian Inference

In [2]:
df = rd.read()
N = np.array(df["N"])
Z = np.array(df["Z"])
BE = np.array(df["binding"])
BE_err = np.array(df["unc_binding"])

In [4]:
def model(a_v,a_s,a_c,a_a,N,Z):
    A = N + Z
    return (a_v*A - a_s*A**(2/3) - a_c*Z*(Z-1)/(A**(1/3)) - a_a*((N-Z)**2)/A)/A

In [5]:
# Parameters
param_names = ['a_v', 'a_s', 'a_c', 'a_a']

def loglikelihood(params):
    (a_v, a_s, a_c, a_a) = params
    log_like = -np.sum(((model(a_v, a_s, a_c, a_a, N, Z) - BE)/BE_err)**2 + np.log(2*np.pi*BE_err**2)) / 2 
    return log_like
    
def prior(u):
    params = u.copy()
    params[0] = scipy.stats.norm.ppf(u[0], 16.0, 5.)
    params[1] = scipy.stats.norm.ppf(u[1], 16.0, 5.)
    params[2] = scipy.stats.norm.ppf(u[2], 0.7, 0.3)
    params[3] = u[3] * 40 
    return params
    

In [6]:
sampler = ultranest.ReactiveNestedSampler(param_names, loglikelihood, prior)

In [7]:
result = sampler.run()
sampler.print_results()
cornerplot(result);

[ultranest] Sampling 400 live points from prior ...


/opt/anaconda3/lib/python3.12/site-packages/ultranest/integrator.py:1903: UserWarning: Sampling from region seems inefficient (0/40 accepted in iteration 2500). To improve efficiency, modify the transformation so that the current live points are ellipsoidal, or use a stepsampler, or set frac_remain to a lower number (e.g., 0.5) to terminate earlier.
  u, v, logl, nc, quality = self._refill_samples(Lmin, ndraw, nit)


KeyboardInterrupt: 